In [1]:
import pandas as pd
film_details_df = pd.read_csv('/mnt/c/Users/ouyangkang/Desktop/films_info.csv', encoding='gbk')

In [2]:
import re
import random

# set seed to reproduce result
random.seed(1026)

actor_list = []
for i in film_details_df.Actors:
    actors = re.sub(r'\'', '', i).split(',')
    actor_list += [i.strip(' ') for i in actors]

actor_list = list(set(actor_list))

ac_dict = {}
uni_code = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
for actor in actor_list:
    ac_dict[actor] = 'AC' + str(random.randint(100, 999)) + random.choice(uni_code) + str(random.randint(10, 99))

In [3]:
len(set(ac_dict.values())) == len(actor_list)

True

In [4]:
pd.DataFrame(ac_dict.items(), columns=['Actor', 'Unicode']).to_csv('/mnt/c/Users/ouyangkang/Desktop/actor_unicode.csv', index=None, encoding='gbk')

In [5]:
dire_list = list(set(film_details_df.Director.tolist()))

random.seed(1026)
dire_dict = {}

for director in dire_list:
    dire_dict[director] = 'DI' + str(random.randint(100, 999)) + random.choice(uni_code) + str(random.randint(10, 99))

print(len(set(dire_dict.values())) == len(dire_list))

pd.DataFrame(dire_dict.items(), columns=['Director', 'Unicode']).to_csv('/mnt/c/Users/ouyangkang/Desktop/director_unicode.csv', index=None, encoding='gbk')

True


In [6]:
# can't directly use dictory
f_d_a_dict = {}
for film in film_details_df.iterrows():
    actors = re.sub(r'\'', '', film[1].Actors).split(',')
    actors = [i.strip(' ') for i in actors]
    f_d_a_dict[film[1].Film_name] = {film[1].Director: actors}


In [7]:
flat_data = []
for film, term in f_d_a_dict.items():
    for director, actor_list in term.items():
        for actor in actor_list:
            flat_data.append([film, director, actor])

In [8]:
f_d_a_df = pd.DataFrame(flat_data, columns=['Film', 'Director', 'Actors'])
f_d_a_df.to_csv('/mnt/c/Users/ouyangkang/Desktop/film_dir_ac.csv', index=None, encoding='gbk')

In [9]:
film_dict = {}
for film in film_details_df.Film_name.tolist():
    film_dict[film] = 'Film' + str(random.randint(100, 999)) + random.choice(uni_code) + str(random.randint(10, 99))

print(len(set(film_dict.values())) == len(film_details_df.Film_name.tolist()))
pd.DataFrame(film_dict.items(), columns=['Film', 'Unicode']).to_csv('/mnt/c/Users/ouyangkang/Desktop/film_unicode.csv', index=None, encoding='gbk')

True


In [10]:
st_connection = []
for film in film_details_df.iterrows():
    actors = re.sub('\'', '', film[1]['Actors']).split(',')
    actors = [i.strip(' ') for i in actors]
    
    for i in range(len(actors)):
        for k in range(i+1, len(actors)):
            st_connection.append([actors[i], actors[k]])

    

In [11]:
pd.DataFrame(st_connection, columns=['Actor_1', 'Actor_2']).to_csv('/mnt/c/Users/ouyangkang/Desktop/connection.csv', encoding='gbk', index=None)

In [12]:
'''
NEO4J COMMAND


CREATE CONSTRAINT actor_unicode FOR (a:actor) REQUIRE a.unicode IS UNIQUE
CREATE CONSTRAINT director_unicode FOR (d:director) REQUIRE d.unicode IS UNIQUE
CREATE CONSTRAINT film_unicode FOR (f:film) REQUIRE f.unicode IS UNIQUE

LOAD CSV WITH HEADERS FROM 'file:///actor_unicode.csv' AS row MERGE (a:actor {name:row.Actor, unicode:row.Unicode})
LOAD CSV WITH HEADERS FROM 'file:///director_unicode.csv' AS row MERGE (d:director {name:row.Director, unicode:row.Unicode})
LOAD CSV WITH HEADERS FROM 'file:///film_unicode.csv' AS row MERGE (f:film {name:row.Film,unicode:row.Unicode})

LOAD CSV WITH HEADERS FROM 'file:///film_dir_ac.csv' AS row MERGE (d:director {name:row.Director}) MERGE (f:film {name:row.Film}) MERGE (d)-[r:Directs]->(f)
LOAD CSV WITH HEADERS FROM 'file:///film_dir_ac.csv' AS row MERGE (f:film {name:row.Film}) MERGE (a:actor {name:row.Actors}) MERGE (f)-[r:Including]->(a)
LOAD CSV WITH HEADERS FROM 'file:///connection.csv' AS row MERGE (a:actor {name:row.Actor_1}) MERGE (b:actor {name:row.Actor_2}) MERGE (a)-[r:cooperation]-(b)

'''

"\nNEO4J COMMAND\n\n\nCREATE CONSTRAINT actor_unicode FOR (a:actor) REQUIRE a.unicode IS UNIQUE\nCREATE CONSTRAINT director_unicode FOR (d:director) REQUIRE d.unicode IS UNIQUE\nCREATE CONSTRAINT film_unicode FOR (f:film) REQUIRE f.unicode IS UNIQUE\n\nLOAD CSV WITH HEADERS FROM 'file:///actor_unicode.csv' AS row MERGE (a:actor {name:row.Actor, unicode:row.Unicode})\nLOAD CSV WITH HEADERS FROM 'file:///director_unicode.csv' AS row MERGE (d:director {name:row.Director, unicode:row.Unicode})\nLOAD CSV WITH HEADERS FROM 'file:///film_unicode.csv' AS row MERGE (f:film {name:row.Film,unicode:row.Unicode})\n\nLOAD CSV WITH HEADERS FROM 'file:///film_dir_ac.csv' AS row MERGE (d:director {name:row.Director}) MERGE (f:film {name:row.Film}) MERGE (d)-[r:Directs]->(f)\nLOAD CSV WITH HEADERS FROM 'file:///film_dir_ac.csv' AS row MERGE (f:film {name:row.Film}) MERGE (a:actor {name:row.Actors}) MERGE (f)-[r:Including]->(a)\nLOAD CSV WITH HEADERS FROM 'file:///connection.csv' AS row MERGE (a:actor {n